In [1]:
import polars as pl

df_base_peers = pl.read_excel("C:/Users/VHENEQUIM/Documents/Estudo/offshorePeer/basePeers.xlsx")
df_base_peers_new = pl.read_excel("C:/Users/VHENEQUIM/Documents/Estudo/offshorePeer/basePeers_teste.xlsx")


In [10]:
df_base_peers_new.group_by("Ativo").agg(pl.len()).filter(pl.col("Ativo").str.contains("Invest. Ext.")).write_csv("base_peers_new_invest_ext.csv")

In [11]:
df_base_peers.group_by("Ativo").agg(pl.len()).filter(pl.col("Ativo").str.contains("Invest. Ext.")).write_csv("base_peers_old_invest_ext.csv")

In [27]:
import json
import polars as pl

# Load the mapping file
with open("mapping_new.json", "r") as f:
    mapping_data = json.load(f)

# Create a lookup dictionary for easier access
investment_lookup = {}
for investment in mapping_data["investments"]:
    investment_lookup[investment["investment_name"]] = {
        "ticker": investment["ticker"],
        "company_country": investment["company_country"],
        "area_of_work": investment["area_of_work"]
    }

# Create a function to map values and handle the prefix
def map_investment_details(ativo):
    # Remove prefix if present
    clean_name = ativo.replace("Invest. Ext. - ", "") if "Invest. Ext. - " in ativo else ativo
    
    # Return investment details if found, otherwise return N/A
    if clean_name in investment_lookup:
        return investment_lookup[clean_name]
    else:
        return {"ticker": "N/A", "company_country": "N/A", "area_of_work": "N/A"}

# Apply mapping to the DataFrame
df_base_peers_new = df_base_peers_new.with_columns([
    pl.col("Ativo").map_elements(lambda x: map_investment_details(x).get("ticker", "N/A")).alias("ticker"),
    pl.col("Ativo").map_elements(lambda x: map_investment_details(x).get("company_country", "N/A")).alias("company_country"),
    pl.col("Ativo").map_elements(lambda x: map_investment_details(x).get("area_of_work", "N/A")).alias("area_of_work"),
    pl.when(pl.col("Ativo").str.contains("Invest. Ext."))
      .then(pl.lit("Invest. Ext."))
      .otherwise(pl.lit(""))
      .alias("investment_type")
])


# Preview the results with new columns
df_base_peers_new.write_excel("base_peers_new_mapped.xlsx")

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


In [29]:
import json
import polars as pl

# Load the mapping file
with open("mapping.json", "r") as f:
    mapping_data = json.load(f)

# Create a comprehensive lookup dictionary that includes all possible names
investment_lookup = {}
for investment in mapping_data["investments"]:
    # Add the primary name
    primary_name = investment["investment_name"]
    details = {
        "ticker": investment["ticker"],
        "company_country": investment["company_country"],
        "area_of_work": investment["area_of_work"]
    }
    investment_lookup[primary_name] = details
    
    # Add all alternate names (if any) pointing to the same details
    if "alternate_names" in investment:
        for alt_name in investment["alternate_names"]:
            investment_lookup[alt_name] = details

# The rest of your code remains the same
def map_investment_details(ativo):
    clean_name = ativo.replace("Invest. Ext. - ", "") if "Invest. Ext. - " in ativo else ativo
    
    if clean_name in investment_lookup:
        return investment_lookup[clean_name]
    else:
        return {"ticker": "N/A", "company_country": "N/A", "area_of_work": "N/A"}

# Apply mapping to the DataFrame
df_base_peers_new = df_base_peers_new.with_columns([
    pl.col("Ativo").map_elements(lambda x: map_investment_details(x).get("ticker", "N/A")).alias("ticker"),
    pl.col("Ativo").map_elements(lambda x: map_investment_details(x).get("company_country", "N/A")).alias("company_country"),
    pl.col("Ativo").map_elements(lambda x: map_investment_details(x).get("area_of_work", "N/A")).alias("area_of_work"),
    pl.when(pl.col("Ativo").str.contains("Invest. Ext."))
      .then(pl.lit("Invest. Ext."))
      .otherwise(pl.lit(""))
      .alias("investment_type")
])


df_base_peers_new.write_excel("base_peers_new_mapped.xlsx")

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


In [31]:
import json
import polars as pl

# Load the mapping file
with open("mapping.json", "r") as f:
    mapping_data = json.load(f)

# Create a comprehensive lookup dictionary that includes all possible names
investment_lookup = {}
for investment in mapping_data["investments"]:
    # Add the primary name
    primary_name = investment["investment_name"]  # or primary_name if you've updated your JSON
    details = {
        "ticker": investment["ticker"],
        "company_country": investment["company_country"],
        "area_of_work": investment["area_of_work"]
    }
    investment_lookup[primary_name] = details
    
    # Add all alternate names (if any) pointing to the same details
    if "alternate_names" in investment:
        for alt_name in investment["alternate_names"]:
            investment_lookup[alt_name] = details

# Create a function to map values and handle multiple prefixes
def map_investment_details(ativo):
    # Handle different prefixes
    clean_name = ativo
    if "Invest. Ext. - " in ativo:
        clean_name = ativo.replace("Invest. Ext. - ", "")
    elif "BDR - " in ativo:
        clean_name = ativo.replace("BDR - ", "")
    
    # Return investment details if found, otherwise return N/A
    if clean_name in investment_lookup:
        return investment_lookup[clean_name]
    else:
        return {"ticker": "N/A", "company_country": "N/A", "area_of_work": "N/A"}

# Determine if an asset is in the investment lookup
def is_in_lookup(ativo):
    clean_name = ativo
    if "Invest. Ext. - " in ativo:
        clean_name = ativo.replace("Invest. Ext. - ", "")
    elif "BDR - " in ativo:
        clean_name = ativo.replace("BDR - ", "")
    
    return clean_name in investment_lookup

# Apply mapping to the DataFrame
df_base_peers_new = df_base_peers_new.with_columns([
    pl.col("Ativo").map_elements(lambda x: map_investment_details(x).get("ticker", "N/A")).alias("ticker"),
    pl.col("Ativo").map_elements(lambda x: map_investment_details(x).get("company_country", "N/A")).alias("company_country"),
    pl.col("Ativo").map_elements(lambda x: map_investment_details(x).get("area_of_work", "N/A")).alias("area_of_work"),
    # Complex logic for investment_type
    pl.when(pl.col("Ativo").str.contains("Invest. Ext."))
      .then(pl.lit("Invest. Ext."))
      .when(pl.col("Ativo").str.contains("BDR -"))
      .then(pl.lit("BDR"))
      .when(pl.col("Ativo").map_elements(is_in_lookup))
      .then(pl.lit("BDR"))
      .otherwise(pl.lit(""))
      .alias("investment_type")
])

# Preview the results with new columns

# Save the results
df_base_peers_new.write_excel("base_peers_new_mapped.xlsx")

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


In [35]:
import json
import polars as pl

# Load the mapping file
with open("mapping.json", "r") as f:
    mapping_data = json.load(f)

# Create two lookup dictionaries - one for exact matches and one for partial matching
exact_lookup = {}
all_investments = []  # List of tuples (name, details) for partial matching

for investment in mapping_data["investments"]:
    details = {
        "ticker": investment["ticker"],
        "company_country": investment["company_country"],
        "area_of_work": investment["area_of_work"]
    }
    
    # Store primary name for both exact and partial matching
    primary_name = investment["investment_name"]  # or primary_name if updated
    exact_lookup[primary_name] = details
    all_investments.append((primary_name, details))
    
    # Store alternate names if any
    if "alternate_names" in investment:
        for alt_name in investment["alternate_names"]:
            exact_lookup[alt_name] = details
            all_investments.append((alt_name, details))

print(all_investments)
# Function to map investment details with both exact and partial matching
def map_investment_details(ativo):
    # Remove prefixes
    clean_name = ativo
    if "Invest. Ext. - " in ativo:
        clean_name = ativo.replace("Invest. Ext. - ", "")
    elif "BDR - " in ativo:
        clean_name = ativo.replace("BDR - ", "")
    
    # First try exact match (faster)
    if clean_name in exact_lookup:
        return exact_lookup[clean_name]
    
    # Then try partial matching
    for name, details in all_investments:
        if name == "MELI":
            print(name, clean_name)
        if name in clean_name:  # Partial match - name is contained in clean_name
            return details
    
    # No match found
    return {"ticker": "N/A", "company_country": "N/A", "area_of_work": "N/A"}

# Similar function to check if an asset is in the lookup
def is_in_lookup(ativo):
    clean_name = ativo
    if "Invest. Ext. - " in ativo:
        clean_name = ativo.replace("Invest. Ext. - ", "")
    elif "BDR - " in ativo:
        clean_name = ativo.replace("BDR - ", "")
    
    # Exact match
    if clean_name in exact_lookup:
        return True
    
    # Partial match
    for name, _ in all_investments:
        if name in clean_name:
            return True
    
    return False

# Apply mapping to the DataFrame
df_base_peers_new = df_base_peers_new.with_columns([
    pl.col("Ativo").map_elements(lambda x: map_investment_details(x).get("ticker", "N/A"), return_dtype=pl.Utf8).alias("ticker"),
    pl.col("Ativo").map_elements(lambda x: map_investment_details(x).get("company_country", "N/A"), return_dtype=pl.Utf8).alias("company_country"),
    pl.col("Ativo").map_elements(lambda x: map_investment_details(x).get("area_of_work", "N/A"), return_dtype=pl.Utf8).alias("area_of_work"),
    # Complex logic for investment_type
    pl.when(pl.col("Ativo").str.contains("Invest. Ext."))
      .then(pl.lit("Invest. Ext."))
      .when(pl.col("Ativo").str.contains("BDR -"))
      .then(pl.lit("BDR"))
      .when(pl.col("Ativo").map_elements(is_in_lookup, return_dtype=pl.Boolean))
      .then(pl.lit("BDR"))
      .otherwise(pl.lit(""))
      .alias("investment_type")
])

# Save the results
df_base_peers_new.write_excel("base_peers_new_mapped.xlsx")

[('UNIVERSAL MUSIC GROUP NV', {'ticker': 'Not Explicitly Provided', 'company_country': 'Netherlands', 'area_of_work': 'Music and Entertainment'}), ('3BRAVALO - TRUXT BRAZIL VALOR - - CITCO - 1498303 VENC.: 26/12/2050', {'ticker': 'Not Explicitly Provided', 'company_country': 'Brazil', 'area_of_work': 'Investment Fund'}), ('GRAYSCALE BITCOIN TRUST BTC', {'ticker': 'GBTC', 'company_country': 'USA', 'area_of_work': 'Cryptocurrency Investment'}), ('SHP FUND SPC CLASS C.', {'ticker': 'Not Explicitly Provided', 'company_country': 'Cayman Islands', 'area_of_work': 'Investment Fund'}), ('MELI US - MERCADOLIBRE INC - 262 VENC.: 31/12/2049', {'ticker': 'MELI', 'company_country': 'Argentina', 'area_of_work': 'E-commerce and Technology'}), ('VIST US - VISTA OIL & GAS SAB DE CV - 25300 VENC.: 31/12/2049', {'ticker': 'VIST', 'company_country': 'Mexico', 'area_of_work': 'Oil and Gas'}), ('AFYA US - AFYA LTD-CLASS A - 31200 VENC.: 31/12/2049', {'ticker': 'AFYA', 'company_country': 'Brazil', 'area_of_w

In [42]:
import json
import polars as pl

# Load the mapping file
with open("mapping.json", "r") as f:
    mapping_data = json.load(f)

# Create two lookup dictionaries - one for exact matches and one for partial matching
exact_lookup = {}
all_investments = []  # List of tuples (name, details) for partial matching

for investment in mapping_data["investments"]:
    details = {
        "ticker": investment["ticker"],
        "company_country": investment["company_country"],
        "area_of_work": investment["area_of_work"]
    }
    
    # Store primary name for both exact and partial matching
    primary_name = investment["investment_name"]
    exact_lookup[primary_name] = details
    all_investments.append((primary_name, details))
    
    # Store alternative names if any
    if "alternative_names" in investment:
        for alt_name in investment["alternative_names"]:
            exact_lookup[alt_name] = details
            all_investments.append((alt_name, details))

# Function to map investment details with both exact and partial matching
def map_investment_details(ativo):
    # Remove prefixes
    clean_name = ativo
    if "Invest. Ext. - " in ativo:
        clean_name = ativo.replace("Invest. Ext. - ", "")
    elif "BDR - " in ativo:
        clean_name = ativo.replace("BDR - ", "")
    
    # First try exact match (faster)
    if clean_name in exact_lookup:
        return exact_lookup[clean_name]
    
    # Then try partial matching
    for name, details in all_investments:
        if name in clean_name:  # Partial match - name is contained in clean_name
            return details
    
    # No match found
    return {"ticker": "N/A", "company_country": "N/A", "area_of_work": "N/A"}

# Similar function to check if an asset is in the lookup
def is_in_lookup(ativo):
    clean_name = ativo
    if "Invest. Ext. - " in ativo:
        clean_name = ativo.replace("Invest. Ext. - ", "")
    elif "BDR - " in ativo:
        clean_name = ativo.replace("BDR - ", "")
    
    # Exact match
    if clean_name in exact_lookup:
        return True
    
    # Partial match
    for name, _ in all_investments:
        if name in clean_name:
            return True
    
    return False

# Apply mapping to the DataFrame
df_base_peers_new = df_base_peers_new.with_columns([
    pl.col("Ativo").map_elements(lambda x: map_investment_details(x).get("ticker", "N/A"), return_dtype=pl.Utf8).alias("ticker"),
    pl.col("Ativo").map_elements(lambda x: map_investment_details(x).get("company_country", "N/A"), return_dtype=pl.Utf8).alias("company_country"),
    pl.col("Ativo").map_elements(lambda x: map_investment_details(x).get("area_of_work", "N/A"), return_dtype=pl.Utf8).alias("area_of_work"),
    # Complex logic for investment_type
    pl.when(pl.col("Ativo").str.contains("Invest. Ext."))
      .then(pl.lit("Invest. Ext."))
      .when(pl.col("Ativo").str.contains("BDR -"))
      .then(pl.lit("BDR"))
      .when(pl.col("Ativo").map_elements(is_in_lookup, return_dtype=pl.Boolean))
      .then(pl.lit("BDR"))
      .otherwise(pl.lit(""))
      .alias("investment_type")
])


# Save the results
df_base_peers_new.write_excel("base_peers_new_mapped.xlsx")